# Power in bands around ripple band (150-250 Hz) during sleep deprivation
- It seems that power in all three bands (100-140, 140-250, 250-400) increases during sleep deprivation.
Does this signify hippocampus is more desynchronized during sleep deprivation?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import subjects
from neuropy.plotting import Fig
from scipy import stats
from neuropy.utils import signal_process

sessions = subjects.sd.allsess# + subjects.sd.utkuAG_day1 + subjects.sd.utkuAG_day2
# sessions = subjects.Nsd().allsess
print(sessions)

In [ ]:
df = pd.DataFrame()
chans = [11, 113, 117, 128, 107]
for sub, sess in enumerate(sessions):

    post = sess.paradigm["post"]
    sd = [post[0], post[0] + 6 * 3600]

    good_channels = sess.probegroup.get_connected_channels(groupby="shank")
    signal = sess.eegfile.get_signal(chans[sub], t_start=sd[0], t_stop=sd[1])
    spect = signal_process.SpectrogramBands(
        signal=signal, window=5, overlap=2, smooth=10
    )
    time = (spect.time - signal.t_start) / 3600
    bp = lambda f1, f2: stats.zscore(spect.get_band_power(f1, f2))
    df = df.append(
        pd.DataFrame(
            {
                "time": time,
                "100-140": bp(100, 140),
                "140-250": bp(140, 250),
                "250-400": bp(250, 400),
                "sub": sub,
            }
        )
    )


In [ ]:
%matplotlib inline
figure = Fig()
fig,gs = figure.draw(grid=(1,1),style='Pres',size=(4,4))

# df = df.groupby('time')
ax = plt.subplot(gs[0])
sns.lineplot(data=df,x='time',y='100-140',ci='sd',ax=ax,label='100-140 Hz')
sns.lineplot(data=df,x='time',y='140-250',ci='sd',ax=ax,label='140-250 Hz')
sns.lineplot(data=df,x='time',y='250-400',ci='sd',ax=ax,label='250-400 Hz')
ax.legend()
ax.set_ylabel('Normalized power')
ax.set_xlabel('Time (ZT)')
ax.set_ylim([-5,6])

In [ ]:
sess.recinfo.channel_groups

# Ripple stats during sleep deprivation
- plotting with linear regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
from neuropy.plotUtil import Fig
import subjects
import seaborn as sns

sessions = subjects.sd.allsess# + subjects.sd.utkuAG_day1 + subjects.sd.utkuAG_day2
# sessions = subjects.Nsd().allsess
print(sessions)


In [ ]:
%matplotlib widget
figure = Fig()
fig,gs = figure.draw(grid=(3,2),style='Pres')
df = pd.DataFrame()
for sub,sess in enumerate(sessions):

    post = sess.paradigm['post']
    # pre = sess.paradigm.pre
    # sd_period = [post[0],post[1]]
    # sd_period = [post[0],post[0]+9*3600]
    try:
        sd_period = sess.paradigm['sd']
    except:
        sd_period = [post[0],post[0]+5*3600]
    rpls = sess.ripple
    # states = sess.brainstates.epochs

    binsz = 600
    sd_bin = np.arange(sd_period[0],sd_period[1]+binsz,binsz)
    hist_rpl = np.histogram(rpls.starts,bins=sd_bin)[0]

    nrem_percent = []
    for start in sd_bin[:-1]:
        prop = sess.brainstates.proportion([start,start+binsz])
        nrem_percent.append(prop['nrem'])

    df = df.append(pd.DataFrame({'n_ripples':hist_rpl/binsz,'tbin':(sd_bin[:-1]-sd_bin[0])/3600,'nrem':nrem_percent,'sub':sub}))


ax = plt.subplot(gs[0])
rpls_mean = df.groupby('tbin').mean().n_ripples
rpls_sem = df.groupby('tbin').sem(ddof=0).n_ripples
# ax.fill_between(rpls_mean.index,rpls_mean-rpls_sem,rpls_mean+rpls_sem,color='#faa49d',alpha=0.6,ec=None)
sns.regplot(x='tbin',y='n_ripples',data=df,ax=ax,color='g',x_estimator=np.mean)
# ax.plot(rpls_mean.index,rpls_mean,color='#faa49d',zorder=2,lw=2)
# ax.axvline(5,ls='--',color='r')
ax.set_xlabel('ZT time (h)')
ax.set_ylabel('Ripple rate')


axsleep= ax.twinx()
color = sess.brainstates.colors['nrem']
sleep_df = df.groupby('tbin').mean()
axsleep.fill_between(sleep_df.index,sleep_df.nrem,color=color,zorder=1,alpha=0.3)
# sns.lineplot(data=df,x='tbin',y='nrem',ax=axsleep,estimator='sum')
axsleep.set_ylim([0,1])
axsleep.spines['right'].set_visible(True)
axsleep.set_ylabel('sleep proportion')

#--- individual sessions -------
# ax = plt.subplot(gs[2])
# # sns.lineplot(x='tbin',y='n_ripples',hue='sub',data=df,ax=ax,legend=None,palette='Blues')
# sns.lineplot(x='tbin',y='n_ripples',hue='sub',data=df,ax=ax,legend=None,palette='Blues')
# # sns.regplot(x='tbin',y='n_ripples',ci=68,data=df,ax=ax,color='g',x_estimator=np.mean,fit_reg=True)
# ax.set_xlabel('ZT time (h)')
# ax.set_ylabel('Ripple rate')


# axsleep= ax.twinx()
# color = sess.brainstates.colors['nrem']
# sleep_df = df.groupby('tbin').mean()
# axsleep.fill_between(sleep_df.index,sleep_df.nrem,color=color,zorder=1,alpha=0.3)
# # sns.lineplot(data=df,x='tbin',y='nrem',ax=axsleep,estimator='sum')
# axsleep.set_ylim([0,1])
# axsleep.spines['right'].set_visible(True)
# axsleep.set_ylabel('sleep proportion')

In [6]:
import numpy as np
from neuropy_viewer import view_signal
import subjects 

sess = subjects.sd.ratNday1[0]
channels= np.concatenate(sess.recinfo.channel_groups[:2]).astype('int')
signal = sess.eegfile.get_signal(channels[:16])
view_signal(signal,sess.probegroup)